## Mise en place du context

In [2]:
from findspark import init
init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

Le contexte est récupéré si existant et dans l'autre cas il sera créé.

In [3]:
sc = pyspark.SparkContext.getOrCreate() # ça marche pas pour moi avec sparkContext
spark = SparkSession.builder.getOrCreate() # Du coup je fais une sparkSession (ça doit revenir presque au même je pense)

## Chargement des données

In [4]:
import os.path as path
from os import makedirs

In [5]:
data_dir = "res"

In [5]:
if not path.isfile("dac.tar.gz"):
    !wget https://s3-eu-west-1.amazonaws.com/kaggle-display-advertising-challenge-dataset/dac.tar.gz

--2018-11-29 21:50:03--  https://s3-eu-west-1.amazonaws.com/kaggle-display-advertising-challenge-dataset/dac.tar.gz
Résolution de s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)… 52.218.36.34
Connexion à s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.36.34|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 4576820670 (4,3G) [application/x-gzip]
Enregistre : «dac.tar.gz»

dac.tar.gz            0%[                    ]   7,66M   929KB/s    tps 81m 14s^C


In [6]:
if not path.isdir(data_dir):
    makedirs(data_dir)

Le fichier [`dac.tar.gz`](https://s3-eu-west-1.amazonaws.com/kaggle-display-advertising-challenge-dataset/dac.tar.gz) doit se situer dans le même répertoire.

In [7]:
if not path.isfile(data_dir + "/train.txt") or not path.isfile(data_dir + "/test.txt"):
    !tar -xvzf dac.tar.gz -C $data_dir

In [6]:
df = spark.read.csv(data_dir + "/train.txt", header=False, sep="\t")

## Preparation des données

In [7]:
df2 = df.dropna()

In [18]:
print(df.count())
print(df2.count())

45840617
756554


In [8]:
df2.schema

StructType(List(StructField(_c0,StringType,true),StructField(_c1,StringType,true),StructField(_c2,StringType,true),StructField(_c3,StringType,true),StructField(_c4,StringType,true),StructField(_c5,StringType,true),StructField(_c6,StringType,true),StructField(_c7,StringType,true),StructField(_c8,StringType,true),StructField(_c9,StringType,true),StructField(_c10,StringType,true),StructField(_c11,StringType,true),StructField(_c12,StringType,true),StructField(_c13,StringType,true),StructField(_c14,StringType,true),StructField(_c15,StringType,true),StructField(_c16,StringType,true),StructField(_c17,StringType,true),StructField(_c18,StringType,true),StructField(_c19,StringType,true),StructField(_c20,StringType,true),StructField(_c21,StringType,true),StructField(_c22,StringType,true),StructField(_c23,StringType,true),StructField(_c24,StringType,true),StructField(_c25,StringType,true),StructField(_c26,StringType,true),StructField(_c27,StringType,true),StructField(_c28,StringType,true),StructFi

Renommage des colonnes par : label, f1, f2, f3,..., f39 + Conversion des colonnes 0 à 13 inclu en float

In [16]:
def mk_newNameCol():
    res = ['label']
    for i in range(1, 40):
        res.append('f' + str(i))
    return res

new_name = mk_newNameCol()
print(new_name)

df3 = df2.select(*[col(c).alias(new_c) for c, new_c in zip(df2.columns, new_name)])
df3 = df3.select(*(col(c).cast("float") if i < 14 else col(c) for i, c in enumerate(df3.columns)))
df3.head(1) # Ca bug à ce moment là je sais pas pourquoi

['label', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39']


[Row(label=1.0, f1=0.0, f2=127.0, f3=1.0, f4=3.0, f5=1683.0, f6=19.0, f7=26.0, f8=17.0, f9=475.0, f10=0.0, f11=9.0, f12=0.0, f13=3.0, f14='05db9164', f15='8947f767', f16='11c9d79e', f17='52a787c8', f18='4cf72387', f19='fbad5c96', f20='18671b18', f21='0b153874', f22='a73ee510', f23='ceb10289', f24='77212bd7', f25='79507c6b', f26='7203f04e', f27='07d13a8f', f28='2c14c412', f29='49013ffe', f30='8efede7f', f31='bd17c3da', f32='f6a3e43b', f33='a458ea53', f34='35cd95c9', f35='ad3062eb', f36='c7dc6720', f37='3fdb382b', f38='010f6491', f39='49d68486')]

Afficher les valeurs uniques pour les attributs 14 à 39 inclu

In [18]:
df3.select(*(col(c) for c in df3.columns[14:])).distinct().show() # marche pas

+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|     f14|     f15|     f16|     f17|     f18|     f19|     f20|     f21|     f22|     f23|     f24|     f25|     f26|     f27|     f28|     f29|     f30|     f31|     f32|     f33|     f34|     f35|     f36|     f37|     f38|     f39|
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|8cf07265|38d50e09|0e3e14a7|19fca40f|25c83c98|7e0ccccf|cb35bf0a|0b153874|a73ee510|df2138cf|1b723d3c|5c28d6c6|2e072c0d|1adce6ef|e2c18d5a|fd81fbb4|e5ba7672|582152eb|21ddcdc9|5840adea|2d4c8386|c9d4222a|3a171ecb|eda696dc|001f3601|984e0db0|
|05db9164|1cfdf714|20eac82e|06b65969|43b19349|7e0ccccf|9

In [19]:
df3.schema

StructType(List(StructField(label,FloatType,true),StructField(f1,FloatType,true),StructField(f2,FloatType,true),StructField(f3,FloatType,true),StructField(f4,FloatType,true),StructField(f5,FloatType,true),StructField(f6,FloatType,true),StructField(f7,FloatType,true),StructField(f8,FloatType,true),StructField(f9,FloatType,true),StructField(f10,FloatType,true),StructField(f11,FloatType,true),StructField(f12,FloatType,true),StructField(f13,FloatType,true),StructField(f14,StringType,true),StructField(f15,StringType,true),StructField(f16,StringType,true),StructField(f17,StringType,true),StructField(f18,StringType,true),StructField(f19,StringType,true),StructField(f20,StringType,true),StructField(f21,StringType,true),StructField(f22,StringType,true),StructField(f23,StringType,true),StructField(f24,StringType,true),StructField(f25,StringType,true),StructField(f26,StringType,true),StructField(f27,StringType,true),StructField(f28,StringType,true),StructField(f29,StringType,true),StructField(f30

## Arrêt du context

In [28]:
sc.stop()